In [1]:
from ECHO_modules.make_data_sets import make_data_sets # Import relevant module

In [19]:
import numpy as np
import pandas as pd

In [27]:
pd.set_option('display.max_columns', None) # Show all columns in the dataframe

In [ ]:
candidates = pd.read_csv('../data/ledb_candidatelevel.csv')
county_candidates = candidates[(candidates.office_consolidated.isin(['County Executive','County Legislature',])) & (candidates.year >= 2004) & (candidates.winner == 'win')][['geo_name', 'state_abb', 'year', 'pid_est']].copy()
county_candidates['geo_name'] = county_candidates['geo_name'].str.upper()
county_candidates.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 17595 entries, 8 to 126565
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   geo_name   17576 non-null  object
 1   state_abb  17595 non-null  object
 2   year       17595 non-null  int64 
 3   pid_est    17302 non-null  object
dtypes: int64(1), object(3)
memory usage: 687.3+ KB


In [90]:
candidates[(candidates['geo_name'] == 'albany') & (candidates.year == 2019) & (candidates.winner == 'win')]

,ledb_candid,full_name,firstname,lastname,fips,geo_name,state_abb,office_consolidated,year,month,district,contest,votes,vote_share,n_winners,winner,incumbent,contributor.cfscore,bonica.cid,prob_democrat,prob_republican,pid_est,prob_female,prob_male,gender_est,prob_black,prob_white,prob_hispanic,prob_asian,prob_other,race_est,ballotpedia_url
7922,30587,craig apple,craig,apple,36001,albany,NY,Sheriff,2019,11.0,000,36001_2019_11_albany_NY_Sheriff_000_1,56868.0,1.00,1,win,1.0,NaN,1.105991e+08,1.00,0.00,D,0.0,1.0,M,0.01,0.97,0.01,0.00,0.0,caucasian,NaN
9210,30389,daniel p. mccoy,daniel,mccoy,36001,albany,NY,County Executive,2019,11.0,NaN,36001_2019_11_albany_NY_County Executive_NA_1,50269.0,0.90,1,win,1.0,0.04,1.115751e+08,1.00,0.00,D,0.0,1.0,M,0.01,0.99,0.00,0.00,0.0,caucasian,NaN
65265,4626,"kermit ""bo"" dorough",kermit,dorough,1301052,albany,GA,Mayor,2019,NaN,mayor,1301052_2019_NA_albany_GA_Mayor_mayor_1,4656.0,0.52,1,win,0.0,0.19,3.955999e+09,0.64,0.36,D,0.0,1.0,M,0.05,0.92,0.02,0.00,0.0,caucasian,NaN
90330,30426,christopher smith,christopher,smith,36001,albany,NY,County Legislature,2019,11.0,39,36001_2019_11_albany_NY_County Legislature_39_1,1918.0,1.00,1,win,1.0,NaN,2.962719e+10,1.00,0.00,D,0.0,1.0,M,0.27,0.72,0.01,0.00,0.0,caucasian,NaN
92165,30467,joanne cunningham,joanne,cunningham,36001,albany,NY,County Legislature,2019,11.0,34,36001_2019_11_albany_NY_County Legislature_34_1,1780.0,0.65,1,win,1.0,NaN,NaN,1.00,0.00,D,1.0,0.0,F,0.03,0.96,0.01,0.00,0.0,caucasian,NaN
92282,30517,paul burgdorf,paul,burgdorf,36001,albany,NY,County Legislature,2019,11.0,23,36001_2019_11_albany_NY_County Legislature_23_1,1771.0,1.00,1,win,1.0,0.64,1.109052e+08,0.00,1.00,R,0.0,1.0,M,0.03,0.95,0.03,0.00,0.0,caucasian,NaN
93164,30447,frank commisso,frank,commisso,36001,albany,NY,County Legislature,2019,11.0,11,36001_2019_11_albany_NY_County Legislature_11_1,1710.0,1.00,1,win,1.0,0.04,3.266148e+09,1.00,0.00,D,0.0,1.0,M,0.00,0.98,0.01,0.00,0.0,caucasian,NaN
93844,30402,andrew joyce,andrew,joyce,36001,albany,NY,County Legislature,2019,11.0,9,36001_2019_11_albany_NY_County Legislature_9_1,1663.0,1.00,1,win,1.0,-0.12,3.034128e+10,1.00,0.00,D,0.0,1.0,M,0.00,0.98,0.02,0.00,0.0,caucasian,NaN
94087,30576,william reinhardt,william,reinhardt,36001,albany,NY,County Legislature,2019,11.0,33,36001_2019_11_albany_NY_County Legislature_33_1,1646.0,1.00,1,win,1.0,-0.89,4.031286e+09,1.00,0.00,D,0.0,1.0,M,0.00,1.00,0.00,0.00,0.0,caucasian,NaN
94609,30472,jeffrey kuhn,jeffrey,kuhn,36001,albany,NY,County Legislature,2019,11.0,35,36001_2019_11_albany_NY_County Legislature_35_1,1609.0,0.59,1,win,0.0,-0.05,1.107742e+08,1.00,0.00,D,0.0,1.0,M,0.01,0.99,0.00,0.00,0.0,caucasian,NaN


In [84]:
county_candidates.groupby(['year','geo_name','state_abb']).pid_est.count().sort_values(ascending=False).head(20)

year  geo_name   state_abb
2019  ALBANY     NY           40
2015  ALBANY     NY           39
2004  ALBANY     NY           39
2011  ALBANY     NY           39
2007  ALBANY     NY           39
2020  DANE       WI           36
2018  OUTAGAMIE  WI           36
2016  DANE       WI           36
2018  DANE       WI           36
2010  DANE       WI           36
2020  OUTAGAMIE  WI           35
2012  DANE       WI           35
2014  DANE       WI           34
2020  WINNEBAGO  WI           34
2016  WINNEBAGO  WI           32
2012  MADISON    IL           30
2011  MONROE     NY           30
2018  WINNEBAGO  WI           30
2019  MONROE     NY           30
2015  MONROE     NY           30
Name: pid_est, dtype: int64

In [42]:
geos = county_candidates[['geo_name', 'state_abb']].drop_duplicates().copy().to_dict('split')['data']
print(len(geos))

609


In [45]:
year_range = [2004, 2021] # Define the year range for the analysis
data_sets = make_data_sets(['Toxic Releases', 'RCRA Violations'])

toxic_release_df = pd.DataFrame()
rcra_violations_df = pd.DataFrame()

for g in geos:
    try:
        toxic_releases = data_sets['Toxic Releases'].store_results(
        region_type="County", region_value=g[0], state=g[1], years=year_range,).dataframe
        toxic_release_df = pd.concat([toxic_release_df, toxic_releases], ignore_index=True)
    except:
        print(f"Error retrieving Toxic Releases for {g[0]}, {g[1]}")
        continue
    try:
        rcra_violations = data_sets['RCRA Violations'].store_results(
        region_type="County", region_value=g[0], state=g[1], years=year_range,).dataframe
        rcra_violations_df = pd.concat([rcra_violations_df, rcra_violations], ignore_index=True)
    except:
        print(f"Error retrieving RCRA Violations for {g[0]}, {g[1]}")


Data last modified: 2025-03-30 00:00:00
There were 9263 program records found
Data last modified: 2025-03-30 00:00:00
There were 1186 program records found
There were 602 program records found
There were 1135 program records found
There were 5339 program records found
There were 5182 program records found
There were 3376 program records found
There were 1395 program records found
There were 293 program records found
There were 291 program records found
There were 4269 program records found
There were 3274 program records found
There were 1545 program records found
There were 1905 program records found
There were 1708 program records found
There were 796 program records found
There were 668 program records found
There were 407 program records found
There were 1703 program records found
There were 5949 program records found
There were 11561 program records found
There were 3096 program records found
There were 18233 program records found
There were 6726 program records found
There were 1

In [46]:
toxic_release_df.shape

(371487, 25)

In [47]:
rcra_violations_df.shape

(204777, 24)

In [72]:
toxic_release_df['FAC_COUNTY'] = toxic_release_df['FAC_COUNTY'].apply(lambda x: x.replace(' COUNTY',''))
rcra_violations_df['FAC_COUNTY'] = rcra_violations_df['FAC_COUNTY'].apply(lambda x: x.replace(' COUNTY',''))

rcra_violations_df['REPORTING_YEAR'] = pd.to_datetime(rcra_violations_df.DATE_VIOLATION_DETERMINED).dt.year
# toxic_release_df.to_csv('../data/toxic_releases.csv',)

# rcra_violations_df.to_csv('../data/rcra_violations.csv',)

In [83]:
toxic = toxic_release_df.groupby(['REPORTING_YEAR','UNIT_OF_MEASURE','FAC_COUNTY','FAC_STATE',]).ANNUAL_EMISSION.sum().reset_index().rename(columns={'REPORTING_YEAR': 'year', 'UNIT_OF_MEASURE': 'unit_of_measure', 'FAC_COUNTY': 'county', 'FAC_STATE': 'state', 'ANNUAL_EMISSION': 'annual_toxic_emissions_sum'})
rcra_violations_totals = rcra_violations_df.groupby(['REPORTING_YEAR','FAC_COUNTY','FAC_STATE']).VIOLATION_TYPE.count().reset_index().rename(columns={'REPORTING_YEAR': 'year', 'FAC_COUNTY': 'county', 'FAC_STATE': 'state', 'VIOLATION_TYPE': 'rcra_violations_count'})
rcra_violations_totals.head(1)

rcra_violations_emissions = pd.merge(rcra_violations_totals, toxic, how='left', on=['year','county','state'])
rcra_violations_emissions = pd.merge(rcra_violations_emissions, county_candidates, how='left', left_on=['year','county','state'], right_on=['year','geo_name','state_abb'])
rcra_violations_emissions = rcra_violations_emissions.drop(columns=['geo_name', 'state_abb'])
rcra_violations_emissions.drop_duplicates(inplace=True)
rcra_violations_emissions.dropna(inplace=True)
rcra_violations_emissions.tail(2)

,year,county,state,rcra_violations_count,unit_of_measure,annual_toxic_emissions_sum,pid_est
20720,2021,WHATCOM,WA,15,Pounds,493895.974188,R
20735,2021,WYANDOTTE,KS,18,Pounds,913720.734380,D


In [82]:
rcra_violations_emissions.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 10347 entries, 0 to 20746
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   year                        10347 non-null  int32  
 1   county                      10347 non-null  object 
 2   state                       10347 non-null  object 
 3   rcra_violations_count       10347 non-null  int64  
 4   unit_of_measure             4547 non-null   object 
 5   annual_toxic_emissions_sum  4547 non-null   float64
 6   pid_est                     4700 non-null   object 
dtypes: float64(1), int32(1), int64(1), object(4)
memory usage: 606.3+ KB


In [80]:
rcra_violations_emissions.duplicated().sum()

np.int64(10400)

In [ ]:
# rcra_violations_df.head()

,ACTIVITY_LOCATION,VIOLATION_TYPE,VIOLATION_TYPE_DESC,VIOL_DETERMINED_BY_AGENCY,DATE_VIOLATION_DETERMINED,ACTUAL_RTC_DATE,SCHEDULED_COMPLIANCE_DATE,FAC_DERIVED_HUC,REGISTRY_ID,FAC_NAME,FAC_STREET,FAC_CITY,FAC_SIC_CODES,FAC_STATE,FAC_ZIP,FAC_COUNTY,FAC_NAICS_CODES,FAC_EPA_REGION,FAC_LAT,FAC_LONG,FAC_DERIVED_WBD,FAC_DERIVED_CD113,FAC_PERCENT_MINORITY,FAC_POP_DEN
0,IL,262.D,Standards Applicable to Recordkeeping and Repo...,E,2004-11-16,07/29/2008,NaN,7120004.0,110000845952,SUNRISE ELECTRONICS INC,130 MARTIN LANE,ELK GROVE VILLAGE,3672,IL,60007,COOK,334412,5.0,42.028943,-87.973271,7.120004e+10,8.0,34.521,3116.22
1,IL,262.A,Standards Applicable to Generators of HW: General,E,2004-11-16,07/29/2008,NaN,7120004.0,110000845952,SUNRISE ELECTRONICS INC,130 MARTIN LANE,ELK GROVE VILLAGE,3672,IL,60007,COOK,334412,5.0,42.028943,-87.973271,7.120004e+10,8.0,34.521,3116.22
2,IL,265.CC,Interim Status Standards for Owners and Operat...,E,2004-11-16,07/29/2008,NaN,7120004.0,110000845952,SUNRISE ELECTRONICS INC,130 MARTIN LANE,ELK GROVE VILLAGE,3672,IL,60007,COOK,334412,5.0,42.028943,-87.973271,7.120004e+10,8.0,34.521,3116.22
3,IL,262.C,Standards Applicable to Generators of HW: Pre-...,E,2004-11-16,07/29/2008,NaN,7120004.0,110000845952,SUNRISE ELECTRONICS INC,130 MARTIN LANE,ELK GROVE VILLAGE,3672,IL,60007,COOK,334412,5.0,42.028943,-87.973271,7.120004e+10,8.0,34.521,3116.22
4,IL,262.D,Standards Applicable to Recordkeeping and Repo...,S,2005-05-13,12/22/2005,NaN,7120003.0,110016732543,DOLTON-RIVERDALE SCH DIST 148,325 W 142ND ST,RIVERDALE,NaN,IL,60827,COOK,61111,5.0,41.637090,-87.630540,7.120003e+10,2.0,93.775,4376.99


In [68]:
np.sort(rcra_violations_df.FAC_COUNTY.unique())

array(['ADA', 'ADA COUNTY', 'ADAMS', 'AIKEN', 'ALACHUA', 'ALACHUA COUNTY',
       'ALAMANCE', 'ALAMEDA', 'ALAMEDA COUNTY', 'ALBANY', 'ALBANY COUNTY',
       'ALBEMARLE', 'ALLEGAN', 'ALLEGANY', 'ALLEGHENY',
       'ALLEGHENY COUNTY', 'ALLEN', 'ALLEN COUNTY', 'ANCHORAGE',
       'ANDERSON', 'ANDROSCOGGIN', 'ANNE ARUNDEL', 'ANOKA',
       'ANOKA COUNTY', 'APACHE', 'ARAPAHOE', 'ARLINGTON', 'ASCENSION',
       'ASCENSION PARISH', 'ATLANTIC', 'BALDWIN', 'BALTIMORE',
       'BALTIMORE CITY', 'BALTIMORE COUNTY', 'BARNSTABLE', 'BARTHOLOMEW',
       'BARTHOLOMEW COUNTY', 'BARTOW', 'BARTOW COUNTY', 'BAY',
       'BAY COUNTY', 'BEAUFORT', 'BEAVER', 'BEAVER COUNTY', 'BELL',
       'BELL COUNTY', 'BENTON', 'BERGEN', 'BERKELEY', 'BERKELEY COUNTY',
       'BERKS', 'BERNALILLO', 'BERNALILLO COUNTY', 'BERRIEN',
       'BERRIEN COUNTY', 'BEXAR', 'BEXAR COUNTY', 'BIBB', 'BIBB COUNTY',
       'BLACK HAWK', 'BLACK HAWK COUNTY', 'BLAIR', 'BLAIR COUNTY',
       'BLOUNT', 'BONNEVILLE', 'BOONE', 'BOONE COUNTY',